<a href="https://colab.research.google.com/github/HIsu1231/Cancer_metastasis_prediction_project/blob/main/%EC%95%94%EC%A0%84%EC%9D%B4%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import torch.nn as nn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device('cuda')

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, df_dir):
    self.df = pd.read_csv(df_dir)
    self.l = []
    for i in range(int(len(self.df.columns)/3)):
      for j in range(0,len(self.df),len(self.df)):
        self.l.append([self.df.iloc[j, i*3:3*i+3]])
                                            
  def __len__(self):                                                                                  
    return  len(self.l)

  def __getitem__(self, idx):
    inp = np.array(self.df.iloc[1:, idx*3:3*idx+3])
    inp = inp.astype(np.float32)
    label = np.array(self.df.iloc[0, idx*3])
    label = label.astype(np.float32)
    return inp, label

In [ ]:
dataset = CustomDataset('/content/drive/MyDrive/전처리데이터_4차_추가_제거.csv')

In [ ]:
dataset_size = len(dataset)

train_size = int(dataset_size * 0.8)
test_size = dataset_size-train_size


train_dataset, test_dataset = random_split(dataset,[train_size, test_size])

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=4)
test_dataloader = DataLoader(test_dataset, batch_size=4)

In [ ]:
for batch_idx, samples in enumerate(train_dataloader):
  x_data, y_data = samples
  print(batch_idx, x_data, y_data)
  break

0 tensor([[[2.0020e+05, 6.4000e+00, 4.0000e-01],
         [2.0021e+05, 5.7000e+00, 9.0000e-01],
         [2.0021e+05, 4.7000e+00, 1.6000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[2.1113e+05, 5.0000e+00, 0.0000e+00],
         [2.1120e+05, 4.9000e+00, 0.0000e+00],
         [2.1120e+05, 5.1000e+00, 3.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[2.1072e+05, 4.4000e+00, 1.5000e+00],
         [2.1080e+05, 2.4000e+00, 0.0000e+00],
         [2.1080e+05, 3.3000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[1.1052e+05, 7.9000e+00, 3.0000e+00],
         [1.1052e+05, 0.0000e+00, 1.0000e+00],
         [

In [ ]:
class LSTM(nn.Module):
  def __init__(self, in_size):
    super().__init__()
    self.in_size = in_size
    self.h_size = 32
    self.n_layers = 2

    self.lstm = nn.LSTM(self.in_size, self.h_size, self.n_layers, batch_first=True)
    self.h_n = None
    self.c_n = None

    self.linear = nn.Linear(self.h_size, 1)
    self.acti = nn.Sigmoid()

  def init_states(self, batch_size):
    self.h_n = torch.zeros(self.n_layers, batch_size, self.h_size,device=device)
    self.c_n = torch.zeros(self.n_layers, batch_size, self.h_size,device=device)
  
  def forward(self, x):
    states = (self.h_n, self.c_n)
    out, (self.h_n, self.c_n) = self.lstm(x, states)

    out = out[:, -1,:]
    out = self.linear(out)
    out = self.acti(out)



    return out

In [ ]:
model = LSTM(in_size=3).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
criterion = nn.BCELoss()

In [ ]:
def cal_acc(pred, label):
    # pred_flat = pred.cpu().data.numpy().argmax(axis=1).flatten()
    # # pred_flat = np.argmax(pred, axis=1).flatten()
    # labels_flat = label.flatten()
    # # print(pred_flat)
    # # print(labels_flat)
    # return np.sum(pred_flat == labels_flat)
    acc = 0
    pred = pred.detach().flatten()
    for i in range(len(pred)):
        if pred[i] < 0.5 :
            pred[i] = 0
        else:
            pred[i] = 1
        if pred[i] == label[i]:
            acc += 1
    return acc

In [ ]:
# Train
n_epochs = 50
acc=0
for epoch in range(1, n_epochs+1):
    y_pred = []
    y_test = []
    # train
    i, train_loss, val_loss = 0, 0., 0.
    epoch_acc = 0
    data_cnt = 0
    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        model.init_states(batch_size=x.shape[0])
        out = model(x)
        # print(out.view(-1))
        y = y.float().view(-1, 1)
        # print('-')
        # print(out)
        # print(y)
        # break
        loss = criterion(out, y)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        i += 1
        acc = cal_acc(out, y)
        epoch_acc += acc
        data_cnt += len(y)
    train_loss /= i
    epoch_acc /= data_cnt
    # validate
    with torch.no_grad():
        i = 0
        for x, y in test_dataloader:
            x = x.to(device)
            y = y.to(device)
            y = y.float().view(-1, 1)
            model.init_states(batch_size=x.shape[0])
            out = model(x)
            loss = criterion(out, y)
            val_loss += loss.item()
            i += 1
            y_pred.append(out)
            y_test.append(y)
        val_loss /= i

    print('epoch: {} | train_loss: {:0.5f} | val_loss: {:0.5f} | epoch_acc : {}'.format(epoch, train_loss, val_loss, epoch_acc))

epoch: 1 | train_loss: 0.59738 | val_loss: 0.50856 | epoch_acc : 0.7829181494661922
epoch: 2 | train_loss: 0.54395 | val_loss: 0.50002 | epoch_acc : 0.8042704626334519
epoch: 3 | train_loss: 0.54294 | val_loss: 0.50229 | epoch_acc : 0.8042704626334519
epoch: 4 | train_loss: 0.54326 | val_loss: 0.50305 | epoch_acc : 0.8042704626334519
epoch: 5 | train_loss: 0.54339 | val_loss: 0.50350 | epoch_acc : 0.8042704626334519
epoch: 6 | train_loss: 0.54347 | val_loss: 0.50379 | epoch_acc : 0.8042704626334519
epoch: 7 | train_loss: 0.54353 | val_loss: 0.50398 | epoch_acc : 0.8042704626334519
epoch: 8 | train_loss: 0.54356 | val_loss: 0.50413 | epoch_acc : 0.8042704626334519
epoch: 9 | train_loss: 0.54359 | val_loss: 0.50424 | epoch_acc : 0.8042704626334519
epoch: 10 | train_loss: 0.54362 | val_loss: 0.50432 | epoch_acc : 0.8042704626334519
epoch: 11 | train_loss: 0.54363 | val_loss: 0.50439 | epoch_acc : 0.8042704626334519
epoch: 12 | train_loss: 0.54365 | val_loss: 0.50445 | epoch_acc : 0.804270

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score

print(recall_score(y_teso, y_so))
print(precision_score(y_teso, y_so))
print(f1_score(y_teso, y_so))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/multiclass.py:150: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  y = np.asarray(y)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/multiclass.py:154: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign 

ValueError: ignored

In [ ]:
print(out.detach().flatten())
aut = out.detach().flatten()
print(y)
for i in range(len(aut)):
    if out[i] < 0.5 :
        out[i] = 0
    else:
        out[i] = 1
    if out[i] == y[i]:
        acc += 1
return acc



In [ ]:
def cal_acc(pred, label):
    pred_flat = pred.cpu().data.numpy().argmax(axis=1).flatten()
    # pred_flat = np.argmax(pred, axis=1).flatten()
    labels_flat = label.flatten()
    # print(pred_flat)
    # print(labels_flat)
    return np.sum(pred_flat == labels_flat)